In [2]:
import setup_django
setup_django.init()

In [32]:
from django.conf import settings
from ratings.models import Rating
from movies.models import Movie
import pandas as pd

LINKS_SMALL_CSV = settings.DATA_DIR / "links_small.csv"
LINKS_SMALL_CSV.exists()

True

In [5]:
qs = Rating.objects.all()
missing_movie_ids = []
for instance in qs:
    if instance.content_object is None:
        missing_movie_ids.append(instance.object_id)

_total = len(missing_movie_ids)
total_missing = set(missing_movie_ids)


6392 57175


In [6]:
print(len(total_missing), _total, qs.count())

6392 57175 100004


In [13]:
links_df = pd.read_csv(LINKS_SMALL_CSV)
links_df.head()

,movieId,imdbId,tmdbId
0,1,114709,862.0
1,2,113497,8844.0
2,3,113228,15602.0
3,4,114885,31357.0
4,5,113041,11862.0


In [15]:
ms_df = links_df.copy()[links_df.movieId.isin(total_missing)]
ms_df.head()

,movieId,imdbId,tmdbId
0,1,114709,862.0
3,4,114885,31357.0
6,7,114319,11860.0
7,8,112302,45325.0
8,9,114576,9091.0


In [17]:
ms_df.shape[0] == len(total_missing)

True

In [18]:
def enrich_imdb_col(val):
    val = str(val)
    if len(val) == 7:
        val = "tt" + val
    elif len(val) == 6:
        val = f"tt0{val}"
    elif len(val) == 5:
        val = f"tt00{val}"
    return val

In [20]:
ms_df["tt"] = ms_df["imdbId"].apply(enrich_imdb_col)

In [21]:
ms_df.head()

,movieId,imdbId,tmdbId,tt
0,1,114709,862.0,tt0114709
3,4,114885,31357.0,tt0114885
6,7,114319,11860.0,tt0114319
7,8,112302,45325.0,tt0112302
8,9,114576,9091.0,tt0114576


In [22]:
MOVIES_CSV = settings.DATA_DIR / "movies_metadata.csv"
MOVIES_CSV.exists()

True

In [26]:
movies_cols = ["title", "overview", "release_date", "imdb_id"]
movies_df = pd.read_csv(MOVIES_CSV, usecols=movies_cols)
movies_df.head()

,imdb_id,overview,release_date,title
0,tt0114709,"Led by Woody, Andy's toys live happily in his ...",1995-10-30,Toy Story
1,tt0113497,When siblings Judy and Peter discover an encha...,1995-12-15,Jumanji
2,tt0113228,A family wedding reignites the ancient feud be...,1995-12-22,Grumpier Old Men
3,tt0114885,"Cheated on, mistreated and stepped on, the wom...",1995-12-22,Waiting to Exhale
4,tt0113041,Just when George Banks has recovered from his ...,1995-02-10,Father of the Bride Part II


In [27]:
missing_movies_df = ms_df.merge(movies_df, left_on="tt", right_on="imdb_id")
missing_movies_df.head()

,movieId,imdbId,tmdbId,tt,imdb_id,overview,release_date,title
0,1,114709,862.0,tt0114709,tt0114709,"Led by Woody, Andy's toys live happily in his ...",1995-10-30,Toy Story
1,4,114885,31357.0,tt0114885,tt0114885,"Cheated on, mistreated and stepped on, the wom...",1995-12-22,Waiting to Exhale
2,7,114319,11860.0,tt0114319,tt0114319,An ugly duckling having undergone a remarkable...,1995-12-15,Sabrina
3,8,112302,45325.0,tt0112302,tt0112302,"A mischievous young boy, Tom Sawyer, witnesses...",1995-12-22,Tom and Huck
4,9,114576,9091.0,tt0114576,tt0114576,International action superstar Jean Claude Van...,1995-12-22,Sudden Death


In [29]:
missing_movies_df["id"] = missing_movies_df["movieId"]
missing_movies_df["id_alt"] = missing_movies_df["tmdbId"].apply(lambda x: str(int(x)))
missing_movies_df.head()                                                       

,movieId,imdbId,tmdbId,tt,imdb_id,overview,release_date,title,id,id_alt
0,1,114709,862.0,tt0114709,tt0114709,"Led by Woody, Andy's toys live happily in his ...",1995-10-30,Toy Story,1,862
1,4,114885,31357.0,tt0114885,tt0114885,"Cheated on, mistreated and stepped on, the wom...",1995-12-22,Waiting to Exhale,4,31357
2,7,114319,11860.0,tt0114319,tt0114319,An ugly duckling having undergone a remarkable...,1995-12-15,Sabrina,7,11860
3,8,112302,45325.0,tt0112302,tt0112302,"A mischievous young boy, Tom Sawyer, witnesses...",1995-12-22,Tom and Huck,8,45325
4,9,114576,9091.0,tt0114576,tt0114576,International action superstar Jean Claude Van...,1995-12-22,Sudden Death,9,9091


In [37]:
final_df = missing_movies_df.copy()[["id", "id_alt", "title"]]
final_df["id_alt"] = final_df["id_alt"].astype(str)
final_df.head()

,id,id_alt,title
0,1,862,Toy Story
1,4,31357,Waiting to Exhale
2,7,11860,Sabrina
3,8,45325,Tom and Huck
4,9,9091,Sudden Death


In [33]:
alt_id_list = final_df["id_alt"].to_list()

In [34]:
movies_qs = Movie.objects.filter(id__in=alt_id_list)
movies_qs.count()

6035

In [36]:
from django.forms.models import model_to_dict

In [46]:
movies_qs = Movie.objects.filter(id__in=alt_id_list)

for obj in movies_qs:
    data = final_df.copy()[final_df["id_alt"] == str(obj.id)]
    if data.shape[0] == 1:
        og_model_data = model_to_dict(obj)
        update_data = data.to_dict("records")[0]
        if obj.title == update_data.get("title"):
            # print(og_model_data)
            og_model_data["id"] = update_data["id"]
            new_model_data = {**og_model_data}
            # print(new_model_data)
            obj.delete()
            Movie.objects.create(**new_model_data)

In [48]:
from ratings.tasks import task_update_movie_ratings
task_update_movie_ratings()

Updated 8117 movie ratings in 0:00:07(7.787115097045898s)
